# Deep Learning - Cat2Toaster

### Van: Mart Veldkamp

Origineel uit "Machine Learning is Fun Part 8: How to Intentionally Trick Neural Networks
A Look into the Future of Hacking" , door Adam Geitgey, Aug 16, 2017
https://medium.com/@ageitgey/machine-learning-is-fun-part-8-how-to-intentionally-trick-neural-networks-b55da32b7196
Aangepast voor image outputs and timing

# Inhoudsopgave

1. [Inleiding](#1)
2. [Het probleem](#2)
3. [Library's laden + data ophalen](#3)
    - [Data ophalen](#4)
4. [Laad de afbeelding](#5)
5. [Scale the images](#6)
6. ["De rest"](#7)
7. [Conclusie](#8)


# 1. Inleiding <a class='anchor' id='1'></a>

Deze opdracht is onderdeel van het overkoepelende vak Deep Learning, dit is één van de meerdere opgaves die ik gebruik in dit vak om te laten zien hoe deep learning gebruikt kan worden voor verschillende doeleinde. Al deze opgaves zijn daarom ook op de lossen door middel van Deep Learning.

Alle code in deze opdrachten zijn van mijzelf, gebasseerd op code uit de lessen, of waar ik zelf vast liep gebaseerd op code snippets van het internet. Mocht het toch voorkomen dat er code 1 op 1 is overgenomen, dan staat dit vermeld in de bronnen

# 2. Het probleem <a class='anchor' id='2'></a>

Het probleem dat ik in deze notebook ga behandelen is het zo aanpassen van een foto dat een model denkt dat het iets anders is. In ons geval gaan we proberen een foto van een kat zo aan te passen dat het uiteindelijk gezien wordt door een model als een toaster. 

Dit doen we door middel van onze image door het model te halen, te kijken hoe erg het lijkt op een toaster, en zo de image aan te passen dat het bij de 2e keer door het model heen halen meer lijkt om een toaster. Als we dit de hele tijd blijven doen komt er op een gegeven moment een foto uit die erg lijkt op een toaster volgens het model, maar in realiteit misschien iets veel anders is. Maar hoe weten we nou zeker dat de afbeelding niet super erg verranderd in de tussentijd? 

Dit kunnen we ook instellen. We kunnen meegeven dat terwijl het eerste model onze afbeeldingen aan het verranderen is, dit maar met kleine stapjes gebeurd. Zodat jij en ik nog wel weten dat het een kat is, maar het model niet meer. 

En dit doen we eigenlijk in een loop, we veranderen dus de hele tijd waarde in de afbeelding zodat het model denkt dat het lijkt een broodrooster, maar voor ons nog steeds lijkt op een kat. De foto hierboven is het eindresultaat, ik wilde natuurlijk makkelijk kunnen zeggen dat het een kat is. Maar grappig genoeg het model niet meer.

# 3. Library's laden + data ophalen <a class='anchor' id='3'></a>

In [1]:
import numpy as np
from keras.preprocessing import image
from keras.applications import inception_v3
from keras import backend as K
from datetime import datetime
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

## Data ophalen <a class='anchor' id='4'></a>

Hier laden we een pre-trained image recognition model in, die we uiteindelijk willen gaan foppen. En ter referentie pakken we de eerste en laatste layer van dit neurale netwerk.

In [2]:
model = inception_v3.InceptionV3()

model_input_layer = model.layers[0].input
model_output_layer = model.layers[-1].output

Instructions for updating:
Colocations handled automatically by placer.


# 4. Laad de afbeelding <a class='anchor' id='5'></a>

Hier laden we de afbeelding in, ik heb er voor gekozen om net zoals het voorbeeld gewoon een toaster te kiezen als afbeelding die we uiteindelijk willen hebben, omdat dat volgens mij wel een traditie is binnen deze minor. Dus we kiezen voor te class #859. Als je een andere class wil kan je deze vinden om [deze site](https://gist.github.com/ageitgey/4e1342c10a71981d0b491e1b8227328b).

En daarna laden we de afbeelding die we willen "faken" in met image.load_img. 

In [3]:
object_type_to_fake = 859

img = image.load_img("cat.png", target_size=(299, 299))
original_image = image.img_to_array(img)

# 5. Scale the images <a class='anchor' id='6'></a>

Wat we hier doen is best makkelijk, wat we moeten inzien is dat ons model een waarde tussen de -1 en 1 verwacht (zoals de meeste AI modellen).  

Eerst maken we onze RGB array die een maximale waarde van 255 heeft terug naar tussen de 0 en de 1 door middel van gedeeld door 255 te doen. Dit betekend voor een waarde 255 (wit) dat het een 1 wordt, en een waarde 0 (zwart) dat het een 0 blijft. Dan trekken we daar 0.5 van af om zo een waarde tussen -0.5 en 0.5 te krijgen, in ons geval wordt het 0.5 (wit) en -0.5 (zwart). En tot slot alles *2 om het tussen de -1 en 1 te krijgen.

Wat daarna gebeurd is eigenlijk ook best logisch, omdat zowel Keras als tensorflow een image met een 4D array verwacht moeten we blijkbaar een 4de dimensie aan onze array toevoegen. En dit doen we door middel van np.expand_dims. Deze 4de dimensie wordt gebruikt door Keras voor batch size.

In [4]:
original_image /= 255.
original_image -= 0.5
original_image *= 2.

original_image = np.expand_dims(original_image, axis=0)

# 6. "De rest" <a class='anchor' id='7'></a>

In [5]:
# Pre-calculate the maximum change we will allow to the image
# We'll make sure our hacked image never goes past this so it doesn't look funny.
# A larger number produces an image faster but risks more distortion.
max_change_above = original_image + 0.01
max_change_below = original_image - 0.01

# Create a copy of the input image to hack on
hacked_image = np.copy(original_image)

# How much to update the hacked image in each iteration
learning_rate = 0.1

# Define the cost function.
# Our 'cost' will be the likelihood out image is the target class according to the pre-trained model
cost_function = model_output_layer[0, object_type_to_fake]

# We'll ask Keras to calculate the gradient based on the input image and the currently predicted class
# In this case, referring to "model_input_layer" will give us back image we are hacking.
gradient_function = K.gradients(cost_function, model_input_layer)[0]

# Create a Keras function that we can call to calculate the current cost and gradient
grab_cost_and_gradients_from_model = K.function([model_input_layer, K.learning_phase()], [cost_function, gradient_function])

cost = 0.0
count = 0

startTime = datetime.now()
# In a loop, keep adjusting the hacked image slightly so that it tricks the model more and more
# until it gets to at least 80% confidence
while cost < 0.8:
    # Check how close the image is to our target class and grab the gradients we
    # can use to push it one more step in that direction.
    # Note: It's really important to pass in '0' for the Keras learning mode here!
    # Keras layers behave differently in prediction vs. train modes!
    cost, gradients = grab_cost_and_gradients_from_model([hacked_image, 0])

    # Move the hacked image one step further towards fooling the model
    hacked_image += gradients * learning_rate

    # Ensure that the image doesn't ever change too much to either look funny or to become an invalid image
    hacked_image = np.clip(hacked_image, max_change_below, max_change_above)
    hacked_image = np.clip(hacked_image, -1.0, 1.0)
    count = count + 1

    print("Model's predicted likelihood that the image is a toaster: {:.8}%".format(cost * 100))
    
print("Time taken :", datetime.now() - startTime, "h:m:s ,  number of iterations ", count)

Model's predicted likelihood that the image is a toaster: 0.00071169438%
Model's predicted likelihood that the image is a toaster: 0.00071179952%
Model's predicted likelihood that the image is a toaster: 0.00071191143%
Model's predicted likelihood that the image is a toaster: 0.00071201944%
Model's predicted likelihood that the image is a toaster: 0.00071212394%
Model's predicted likelihood that the image is a toaster: 0.0007122318%
Model's predicted likelihood that the image is a toaster: 0.00071234435%
Model's predicted likelihood that the image is a toaster: 0.00071244699%
Model's predicted likelihood that the image is a toaster: 0.00071255226%
Model's predicted likelihood that the image is a toaster: 0.00071266172%
Model's predicted likelihood that the image is a toaster: 0.00071276972%
Model's predicted likelihood that the image is a toaster: 0.00071287968%
Model's predicted likelihood that the image is a toaster: 0.00071298773%
Model's predicted likelihood that the image is a toa

# Conclusie <a class='anchor' id='8'></a>

Hieronder schrijven we een kleine conclusie kwa de afbeelding output, we printen daarbij de gehackde images om te laten zien hoe weinig zo een image eigenlijk verranderd van de orginele. Wat zoals verwacht super weinig is. Wat leuk is om te zien is dat je als mens nog wel inziet dat dit een kat is, maar jij ook kan zien dat dit een rare foto is met random groene / blauwe pixels.

In [6]:
# De-scale the image's pixels from [-1, 1] back to the [0, 255] range
img = hacked_image[0]
img /= 2.
img += 0.5
img *= 255.
new=image.array_to_img(img)
image.save_img('hacked1.png',new)
# check that image is saved correctly
new1=image.load_img('hacked1.png',new)
print(new1.format)
print(new1.mode)
print(new1.size)
new1.show()
print("Completed\n")

PNG
RGB
(299, 299)
Completed

